In [5]:
import numpy as np
import pandas as pd
from pathlib import Path

In [1]:
#6.3.2 应用简化版SMO算法处理小规模数据集

In [3]:
def loadDataSet(fileName):
    dataMat,labelMat = [],[]
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

In [21]:
def selectJrand(i,m):
    j = i
    while j == i:
        j = int(np.random.uniform(0,m))
    return j

def clipAlpha(aj,H,L):
    if aj > H:
        aj = H
    if aj < L:
        aj = L
    return aj

In [22]:
data_path = Path('D:\\python_algorithm\\machinelearinginaction\\《机器学习实战》Python3代码\\Ch06')
dataArr,labelArr = loadDataSet(data_path / 'testSet.txt')

In [35]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()    #行向量转为列向量
    b = 0
    m,n = np.shape(dataMatrix)
    
    alphas = np.mat(np.zeros((m,1)))
    iter = 0
    
    while iter < maxIter:
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(np.multiply(alphas,labelMat).T * \
                        (dataMatrix*dataMatrix[i,:].T)) + b
            Ei = fXi - float(labelMat[i])
            
            if ((labelMat[i] * Ei) < -toler and alphas[i] < C) or \
                ((labelMat[i] * Ei) > toler and alphas[i] > 0):
                j = selectJrand(i,m)                                 #随机选择另外一个alpha
                
                fXj = float(np.multiply(alphas,labelMat).T * \
                        (dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                
                if labelMat[i] != labelMat[j]:
                    L = max(0,alphas[j] - alphas[i])
                    H = min(C,C+alphas[j] - alphas[i])
                else:
                    L = max(0,alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                    
                if L == H:
                    #print('L == H')
                    continue
                
                eta = 2.0 * dataMatrix[i,:] * dataMatrix[j,:].T - \
                      dataMatrix[i,:] * dataMatrix[i,:].T - \
                      dataMatrix[j,:] * dataMatrix[j,:].T
                
                if eta >= 0:
                    #print('eta >= 0')
                    continue
                
                alphas[j] -= labelMat[j] * (Ei - Ej)/eta                      #求另外一个alpha的值
                alphas[j] = clipAlpha(alphas[j], H, L)
                
                if abs(alphas[j] - alphaJold) < 0.00001:
                    #print('j not moving enough')
                    continue
                
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * \
                    dataMatrix[i,:] * dataMatrix[i,:].T - \
                    labelMat[j] * (alphas[j] - alphaJold) * \
                    dataMatrix[i,:] * dataMatrix[j,:].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * \
                    dataMatrix[i,:] * dataMatrix[j,:].T - \
                    labelMat[j] * (alphas[j] - alphaJold) * \
                    dataMatrix[j,:] * dataMatrix[j,:].T
                
                if alphas[i] > 0 and alphas[i] < C:
                    b = b1
                elif alphas[j] > 0 and alphas[j] < C:
                    b = b2
                else:
                    b = (b1 + b2)/2.0
                
                alphaPairsChanged += 1
                
        if alphaPairsChanged == 0:
            iter += 1
        else:
            iter = 0
        #print('iteration number: %d'%iter)
    return b,alphas
            

In [36]:
b,alphas = smoSimple(dataArr,labelArr,0.6,0.001,40)

In [37]:
b

matrix([[-3.82460044]])

In [38]:
alphas[alphas>0]

matrix([[0.10716938, 0.2567208 , 0.01883409, 0.34505608]])

In [39]:
for i in range(100):
    if alphas[i] > 0.0:
        print (dataArr[i],labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[5.286862, -2.358286] 1.0
[6.080573, 0.418886] 1.0
